<a href="https://colab.research.google.com/github/jsebastianquiroga/Big_Data_Derby_2022_SI/blob/main/Notebooks/Sistemas_Inteligentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img style="float: center; padding-right: 10px;" src="https://www.pikpng.com/pngl/b/467-4670553_universidad-javeriana-esslingen-am-neckar-clipart.png" width="100" height="150">




***Integrantes:***

**Joshep Andersson Blanco Reyes**

**Leonardo García Diaz**

**Sebastián Quiroga Bernal**

# <img style="float: left; padding-right: 10px;" src="https://images7.alphacoders.com/671/671791.jpg"  height="300">




# Big Data Derby 2022

The goal of this competition is to analyze horse racing tactics, drafting strategies, and path efficiency. You will develop a model using never-before-released coordinate data along with basic race information.

Your work will help racing horse owners, trainers, and veterinarians better understand how equine performance and welfare fit together. With better data analysis, equine welfare could significantly improve.

https://www.kaggle.com/competitions/big-data-derby-2022/overview




## ***Paquetes de conexión***

In [4]:

# Conexion a gdrive
from pydrive.auth import GoogleAuth
from google.colab import drive
from pydrive.drive import GoogleDrive
from google.colab import auth
import pandas as pd
from oauth2client.client import GoogleCredentials

In [5]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

nyra_2019_complete = '1AAshxhslPjhxg-UC86f9Z97s3W81nayF' #<-- Key del file en drive (nyra_2019_complete.csv 610 mb)
nyra_race_table    = '1KtYQEvt4pboSqs7bO6jfdCHADo9PMzo9'#<-- Key del file en drive (nyra_race_table.csv 93 kb)
nyra_start_table   = '1LYUx_De3U7TBw6GgqRk6whVzMjuJjSaz'#<-- Key del file en drive (nyra_start_table.csv 722 Kb)
nyra_tracking_table= '15y4AtfLiyop9FWdc0SRaMUs-jKgnOppl'#<-- Key del file en drive (nyra_tracking_table.csv 320,7 mb)


download = drive.CreateFile({'id': nyra_2019_complete}) 
# Download the file to a local disc
download.GetContentFile('nyra_2019_complete.csv')
nyra_2019  = pd.read_csv("nyra_2019_complete.csv", header=None, nrows=10)
nyra_2019.head()


download = drive.CreateFile({'id': nyra_race_table}) 
# Download the file to a local disc
download.GetContentFile('nyra_race_table.csv')
nyra_race  = pd.read_csv("nyra_race_table.csv", header=None, nrows=10)
nyra_race.head()

download = drive.CreateFile({'id': nyra_start_table}) 
# Download the file to a local disc
download.GetContentFile('nyra_start_table.csv')
nyra_start  = pd.read_csv("nyra_start_table.csv", header=None, nrows=10)



download = drive.CreateFile({'id': nyra_start_table}) 
# Download the file to a local disc
download.GetContentFile('nyra_tracking_table.csv')
nyra_tracking  = pd.read_csv("nyra_tracking_table.csv", header=None, nrows=10)

## ***Revisando los heads***

In [6]:
nyra_2019.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,AQU,2019-01-01,9,6,72,40.672902,-73.827607,600,D,GD,48,CLM,25000.0,420,120,Andre Shivnarine Worrie,2090,8
1,AQU,2019-01-01,9,6,73,40.672946,-73.827587,600,D,GD,48,CLM,25000.0,420,120,Andre Shivnarine Worrie,2090,8
2,AQU,2019-01-01,9,6,74,40.672990,-73.827568,600,D,GD,48,CLM,25000.0,420,120,Andre Shivnarine Worrie,2090,8
3,AQU,2019-01-01,9,6,63,40.672510,-73.827781,600,D,GD,48,CLM,25000.0,420,120,Andre Shivnarine Worrie,2090,8
4,AQU,2019-01-01,9,6,64,40.672553,-73.827762,600,D,GD,48,CLM,25000.0,420,120,Andre Shivnarine Worrie,2090,8


In [7]:
nyra_tracking.head()

,0,1,2,3,4,5,6,7
0,AQU,2019-01-01,1,1,123,Dylan Davis,130,2
1,AQU,2019-01-01,1,2,120,Junior Alvarado,295,3
2,AQU,2019-01-01,1,3,118,Jose Lezcano,180,4
3,AQU,2019-01-01,1,4,123,Jomar Garcia,1280,5
4,AQU,2019-01-01,1,5,118,Manuel Franco,1150,1


In [8]:
nyra_start.head()

,0,1,2,3,4,5,6,7
0,AQU,2019-01-01,1,1,123,Dylan Davis,130,2
1,AQU,2019-01-01,1,2,120,Junior Alvarado,295,3
2,AQU,2019-01-01,1,3,118,Jose Lezcano,180,4
3,AQU,2019-01-01,1,4,123,Jomar Garcia,1280,5
4,AQU,2019-01-01,1,5,118,Manuel Franco,1150,1


In [9]:
nyra_race.head(3)

,0,1,2,3,4,5,6,7,8,9
0,track_id,race_date,race_number,distance_id,course_type,track_condition,run_up_distance,race_type,purse,post_time
1,AQU,2019-01-01,1,650,D,MY,36,AOC,80000,01220
2,AQU,2019-01-01,2,600,D,MY,48,MCL,41000,01250


## ***Next Steps***
- Añadir headers
- Ver si hay llaves para unir los datos en un solo datasets (Hacer joins)
- Explorar las variables.
- Definir el Target